In [ ]:
!pip install --quiet tensorflow==1.15
!pip install --quiet "tensorflow_hub>=0.6.0"
!pip3 install --quiet tensorflow_text==1.15

import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-large/3")

# Compute a representation for each message, showing various lengths supported.
word = "Elephant"
sentence = "I am a sentence for which I would like to get its embedding."
paragraph = (
    "Universal Sentence Encoder embeddings also support short paragraphs. "
    "There is no hard limit on how long the paragraph is. Roughly, the longer "
    "the more 'diluted' the embedding will be.")
messages = [word, sentence, paragraph]

tf.logging.set_verbosity(tf.logging.ERROR)

session = tf.Session()
session.run([tf.global_variables_initializer(), tf.tables_initializer()])
message_embeddings = session.run(embed(messages))

for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
  print("Message: {}".format(messages[i]))
  print("Embedding size: {}".format(len(message_embedding)))
  message_embedding_snippet = ", ".join(
      (str(x) for x in message_embedding[:3]))
  print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

     |████████████████████████████████| 412.3MB 34kB/s 
     |████████████████████████████████| 3.8MB 42.0MB/s 
     |████████████████████████████████| 512kB 49.3MB/s 
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
     |████████████████████████████████| 9.1MB 3.1MB/s 
Message: Elephant
Embedding size: 512
Embedding: [0.04498472437262535, -0.057433925569057465, 0.0022114855237305164, ...]

Message: I am a sentence for which I would like to get its embedding.
Embedding size: 512
Embedding: [0.055680178105831146, -0.009607946500182152, 0.006246278528124094, ...]

Message: Universal Sentence Encoder embeddings also support short paragraphs. There is no hard limit on how long the paragraph is. Roughly, the longer the more 'diluted' the embedding will be.
Embedding size: 512
Embedding: [0.038749393075704575, 0.0765201598405838, -0.0007945799734443426, ...]



In [ ]:
print(np.array(session.run(embed(['word']))))

(1, 512)


In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive/


In [ ]:
file = '/gdrive/My Drive/app/data_unstemmed.csv' 
# file = '/gdrive/My Drive/app/train_4k.csv' 
# data = pd.read_csv(file, names=["Ref_paper", "Cit_paper", "Cit_Text", "Ref_text", "label"])
data = pd.read_csv(file)
data = data[['Cit_Text', 'Ref_text', 'label']]
# data = data.sample(10000)
print(len(data[data['label']==1]),len(data[data['label']==0]))
print(data.head())
data['Cit_Text'] = data['Cit_Text'].astype(str)
data['Ref_text'] = data['Ref_text'].astype(str)

33753 29989
                                            Cit_Text  ... label
0  finally use pos tagger trained tweets gimpel e...  ...     1
1  finally use pos tagger trained tweets gimpel e...  ...     1
2  finally use pos tagger trained tweets gimpel e...  ...     0
3  finally use pos tagger trained tweets gimpel e...  ...     0
4  however pos taggers twitter available limited ...  ...     1

[5 rows x 3 columns]


In [ ]:
import pickle
left = []
right = []
for i in range(len(data)):
  left.append(data.loc[i, "Cit_Text"])
  right.append(data.loc[i, "Ref_text"])
  if i % 10000 == 0:
    right_data = session.run(embed(np.array(right)))
    left_data = session.run(embed(np.array(left)))
    
    dbfile = open('/gdrive/My Drive/app/Universal_Store_11/right' + str(i/10000), 'ab')       
    pickle.dump(right_data, dbfile)                      
    dbfile.close()

    dbfile = open('/gdrive/My Drive/app/Universal_Store_11/left' + str(i/10000), 'ab')       
    pickle.dump(left_data, dbfile)                      
    dbfile.close()

    left = []
    right = [] 
    print('Done', i)

Done 0
Done 10000
Done 20000
Done 30000
Done 40000
Done 50000
Done 60000


In [ ]:
print(len(left), len(right))
right_data = session.run(embed(np.array(right)))
left_data = session.run(embed(np.array(left)))

dbfile = open('/gdrive/My Drive/app/Universal_Store_11/right' + str(7), 'ab')       
pickle.dump(right_data, dbfile)                      
dbfile.close()

dbfile = open('/gdrive/My Drive/app/Universal_Store_11/left' + str(7), 'ab')       
pickle.dump(left_data, dbfile)                      
dbfile.close()

left = []
right = [] 
print('Done', i)

3741 3741
Done 63741


In [ ]:
path = '/gdrive/My Drive/app/Universal_Store_11/'
dbfile = open(path + 'left'+str(0)+'.0', 'rb')
left_data = pickle.load(dbfile)
dbfile.close()
dbfile = open(path + 'right'+str(0)+'.0', 'rb')
right_data = pickle.load(dbfile)
dbfile.close()
# left_data = 
for i in range(1, 8):

  dbfile = open(path + 'left'+str(i)+'.0', 'rb')
  left_data = np.concatenate((left_data, pickle.load(dbfile)))
  dbfile.close()
  dbfile = open(path + 'right'+str(i)+'.0', 'rb')
  right_data = np.concatenate((right_data,pickle.load(dbfile)))
  dbfile.close()
  print(left_data.shape, right_data.shape)

(10001, 512) (10001, 512)
(20001, 512) (20001, 512)
(30001, 512) (30001, 512)
(40001, 512) (40001, 512)
(50001, 512) (50001, 512)
(60001, 512) (60001, 512)
(63742, 512) (63742, 512)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
X = left_data - right_data
Y = data['label'].values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,shuffle=False)

In [ ]:
!pip3 install --quiet catboost
print(X.shape, Y.shape)
import catboost as cbt
trainer = []
for a,b in zip(X_train, Y_train):
  trainer.append((a,b))
trainer = np.array(trainer)
np.random.shuffle(trainer)
X_train = []
Y_train = []

for a,b in trainer:
  X_train.append(a)
  Y_train.append(b)
X_train = np.array(X_train)
Y_train = np.array(Y_train)

cbt_model = cbt.CatBoostClassifier()
cbt_model.fit(X_train, Y_train)
y_pred = cbt_model.predict(X_test)
print(classification_report(Y_test, y_pred))

     |████████████████████████████████| 64.8MB 57kB/s 
(63742, 512) (63742,)
Learning rate set to 0.055213
0:	learn: 0.6845939	total: 212ms	remaining: 3m 31s
1:	learn: 0.6770991	total: 342ms	remaining: 2m 50s
2:	learn: 0.6704274	total: 468ms	remaining: 2m 35s
3:	learn: 0.6648512	total: 597ms	remaining: 2m 28s
4:	learn: 0.6607854	total: 738ms	remaining: 2m 26s
5:	learn: 0.6567916	total: 866ms	remaining: 2m 23s
6:	learn: 0.6516325	total: 989ms	remaining: 2m 20s
7:	learn: 0.6478316	total: 1.11s	remaining: 2m 17s
8:	learn: 0.6446218	total: 1.24s	remaining: 2m 17s
9:	learn: 0.6415815	total: 1.37s	remaining: 2m 16s
10:	learn: 0.6387768	total: 1.52s	remaining: 2m 16s
11:	learn: 0.6361153	total: 1.65s	remaining: 2m 16s
12:	learn: 0.6333328	total: 1.78s	remaining: 2m 15s
13:	learn: 0.6307749	total: 1.91s	remaining: 2m 14s
14:	learn: 0.6285570	total: 2.04s	remaining: 2m 13s
15:	learn: 0.6263744	total: 2.16s	remaining: 2m 12s
16:	learn: 0.6243990	total: 2.29s	remaining: 2m 12s
17:	learn: 0.621390

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_train, Y_train)
y_pred = clf.predict(X_test)
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.62      0.63      0.63      6037
           1       0.66      0.65      0.66      6712

    accuracy                           0.64     12749
   macro avg       0.64      0.64      0.64     12749
weighted avg       0.64      0.64      0.64     12749



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
!pip3 install --quiet xgboost
from xgboost import XGBClassifier
x_clf = XGBClassifier().fit(X_train, Y_train)
y_pred = x_clf.predict(X_test)
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.69      0.66      0.67      6037
           1       0.70      0.73      0.72      6712

    accuracy                           0.70     12749
   macro avg       0.70      0.69      0.69     12749
weighted avg       0.70      0.70      0.70     12749

